In [4]:
from ib_insync import *
import pandas as pd
import random
from datetime import date, time

In [5]:

# initiate
util.startLoop() 
ib = IB()

# # connect to client
my_client_id = random.randint(1, 9999)
try:
    ib.connect('127.0.0.1', 7497, clientId=my_client_id)
    print("✅ Connected!")
except Exception as e:
    print(f"❌ Error: {e}")


# get historic prices
ib.reqMarketDataType(3) 

# getting eurostoxx 50 data
indices_config = [
    {"symbol": "ESTX50", "exchange": "EUREX",  "currency": "EUR", "name": "Euro Stoxx 50"},
]

# now iterate over the indices we want to trade
price_data_ls = []
for idx, conf in enumerate(indices_config):
    
    # getting the contract number
    contract = Index(conf["symbol"], conf["exchange"], conf["currency"])
    
    # verify contract existance
    try:
        ib.qualifyContracts(contract)
    except Exception as e:
        print(f"⚠️ Could not qualify {conf['symbol']}: {e}")
        continue
    print(f"Requesting: {contract.symbol} on {contract.exchange}...")

    # get the bars
    try:
        bars = ib.reqHistoricalData(
            contract,
            endDateTime='',
            durationStr='50 Y',
            barSizeSetting='1 day',
            whatToShow='TRADES',
            useRTH=True,
            formatDate=1
        )
    except Exception as e:
        print(f"❌ Failed request for {conf['symbol']}: {e}")
        continue

    # to dataframe
    df = util.df(bars)
    if df is not None and not df.empty:
        df['date'] = pd.to_datetime(df['date'])
        df = df.rename({"date": "datetime"}, axis=1)
        df["date"] = df["datetime"].dt.date
        df["symbol"] = conf["symbol"]
        df["symbol_ib"] = contract.symbol
        df["name"] = conf["name"]
        df["currency"] = conf["currency"]
        df["contract_id"] = contract.conId
        price_data_ls.append(df)
        print(f"✅ SUCCESS - {conf['name']}")
    else:
        print(f"⚠️ No data received for {conf['symbol']}. Check permissions.")

# disconnext
ib.disconnect()

# combine and save
if price_data_ls:
    price_data_df = pd.concat(price_data_ls, ignore_index=True)
    print("\nFinal Data Shape:", price_data_df.shape)
    price_data_df.to_csv("data/eurostoxx.csv", index=False)
else:
    print("No data collected.")

✅ Connected!
Requesting: ESTX50 on EUREX...
✅ SUCCESS - Euro Stoxx 50

Final Data Shape: (5246, 14)


In [6]:
# some basic stats
price_data_df

,datetime,open,high,low,close,volume,average,barCount,date,symbol,symbol_ib,name,currency,contract_id
0,2005-06-29,3129.93,3190.96,3100.92,3177.64,0.0,0.0,3160,2005-06-29,ESTX50,ESTX50,Euro Stoxx 50,EUR,4356500
1,2005-06-30,3177.21,3199.49,3174.34,3185.14,0.0,0.0,2968,2005-06-30,ESTX50,ESTX50,Euro Stoxx 50,EUR,4356500
2,2005-07-01,3171.99,3211.28,3169.98,3210.70,0.0,0.0,3094,2005-07-01,ESTX50,ESTX50,Euro Stoxx 50,EUR,4356500
3,2005-07-04,3212.63,3218.17,3202.93,3215.72,0.0,0.0,2181,2005-07-04,ESTX50,ESTX50,Euro Stoxx 50,EUR,4356500
4,2005-07-05,3215.21,3216.57,3190.16,3212.59,0.0,0.0,3268,2005-07-05,ESTX50,ESTX50,Euro Stoxx 50,EUR,4356500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5241,2026-01-26,5948.97,5971.36,5926.30,5957.80,0.0,0.0,2019,2026-01-26,ESTX50,ESTX50,Euro Stoxx 50,EUR,4356500
5242,2026-01-27,5972.97,6004.54,5960.02,5994.59,0.0,0.0,2017,2026-01-27,ESTX50,ESTX50,Euro Stoxx 50,EUR,4356500
5243,2026-01-28,5988.60,6024.55,5933.20,5933.20,0.0,0.0,2027,2026-01-28,ESTX50,ESTX50,Euro Stoxx 50,EUR,4356500
5244,2026-01-29,5963.64,5973.25,5879.16,5891.95,0.0,0.0,2027,2026-01-29,ESTX50,ESTX50,Euro Stoxx 50,EUR,4356500
